## Import Libraries

In [1]:
from pyspark.sql.functions import hour,minute,second,col,avg,when
from pyspark.sql import SQLContext
import pyspark.sql.functions as sqlFun
import datetime
sqlContext = SQLContext(sc)

## Read csv file from hdfs server

In [2]:
df_log = sqlContext.read.csv("hdfs:///user/spark-file/output/all",header = True, inferSchema = True)
df_log.show(10)

+--------------------+-------------------+-------------------+-------------------+-------------------+
|           user_name|         start_time|           end_time|          idle_time|       working_hour|
+--------------------+-------------------+-------------------+-------------------+-------------------+
|prathameshsalap@g...|2019-10-21 08:35:01|2019-10-21 19:25:02|2019-10-21 01:50:00|2019-10-21 09:00:01|
|vaishusawant143@g...|2019-10-21 08:35:01|2019-10-21 19:25:02|2019-10-21 01:35:00|2019-10-21 09:15:01|
|     you@example.com|2019-10-21 08:35:01|2019-10-21 19:25:01|2019-10-21 01:55:00|2019-10-21 08:55:00|
|  shelkeva@gmail.com|2019-10-21 08:50:02|2019-10-21 19:25:02|2019-10-21 01:00:00|2019-10-21 09:35:00|
|ashutoshrit64@gma...|2019-10-21 08:50:01|2019-10-21 19:25:02|2019-10-21 00:30:00|2019-10-21 10:05:01|
|akshaypatwari24@g...|2019-10-21 09:00:01|2019-10-21 19:15:01|2019-10-21 01:30:00|2019-10-21 08:45:00|
|sushantpatwari123...|2019-10-21 09:05:01|2019-10-21 19:00:01|2019-10-21 

In [3]:
print('Rows : ',df_log.count(), 'Columns : ',len(df_log.columns))    # total no of row and column

Rows :  307 Columns :  5


### Find total number of hours for each user in idle hours 

In [4]:
df_idle = df_log.drop('working_hour', 'start_time', 'end_time')

In [7]:
df_avg = df_idle.groupBy('user_name').agg(sqlFun.from_unixtime(sqlFun.avg(sqlFun.unix_timestamp('idle_time')),'hh:mm:ss').alias('avg_time'))
df_avg.show(5,truncate=50)

+---------------------------+--------+
|                  user_name|avg_time|
+---------------------------+--------+
|    ashutoshrit64@gmail.com|01:11:15|
|giridhardandikwar@gmail.com|08:48:20|
|  priyankagorte95@gmail.com|12:22:30|
|              nikitapawar17|03:03:20|
|   salinabodale73@gmail.com|02:20:00|
+---------------------------+--------+
only showing top 5 rows



In [8]:
df_avg_hours = df_avg.withColumn('avg_hour', (hour(df_avg['avg_time'])*3600 + minute(df_avg['avg_time'])*60 + second(df_avg['avg_time']))/3600)
df_avg_hours.show(5,truncate=50)

+---------------------------+--------+------------------+
|                  user_name|avg_time|          avg_hour|
+---------------------------+--------+------------------+
|    ashutoshrit64@gmail.com|01:11:15|            1.1875|
|giridhardandikwar@gmail.com|08:48:20| 8.805555555555555|
|  priyankagorte95@gmail.com|12:22:30|            12.375|
|              nikitapawar17|03:03:20|3.0555555555555554|
|   salinabodale73@gmail.com|02:20:00|2.3333333333333335|
+---------------------------+--------+------------------+
only showing top 5 rows



In [9]:
#calculating average hours
total_avg_work_hour = df_avg_hours.select(avg('avg_hour'))
total_avg_work_hour.show()

+-----------------+
|    avg(avg_hour)|
+-----------------+
|5.535307017543859|
+-----------------+



In [10]:
avg_count = df_avg_hours.filter(sqlFun.col("avg_hour") > 5.535307017543859)
avg_count.count()

34

In [11]:
df_avg_hours.filter(df_avg_hours['avg_hour'] > 5.535307017543859).select('user_name').show(5,truncate=50)

+--------------------------------+
|                       user_name|
+--------------------------------+
|     giridhardandikwar@gmail.com|
|       priyankagorte95@gmail.com|
|    mail2anik.official@gmail.com|
|“shivnajalisangale5456@gmail.com|
|    samruddhichitnis02@gmail.com|
+--------------------------------+
only showing top 5 rows

